# 🌟 ZTF Lightcurve Downloader via ALeRCE API
该 notebook 会：
1. 从 `tns_search.csv` 中提取 ZTF objectId
2. 通过 ALeRCE Python SDK 下载 g/r 波段光变曲线
3. 保存每个超新星到独立文件夹，包含：g.csv、r.csv、label.txt

In [ ]:
# 如未安装，请先执行：
# pip install alerce pandas

In [5]:
import pandas as pd
from pathlib import Path
from alerce.core import Alerce
import time

In [2]:
def extract_ztf(internal_names):
    try:
        parts = [x.strip() for x in internal_names.split(',')]
        return next((name for name in parts if name.startswith('ZTF')), None)
    except:
        return None

In [3]:
# 加载 TNS CSV（请放在 notebook 同一目录）
tns_df = pd.read_csv("tns_supernovae_only.csv")
ztf_df = tns_df[tns_df["internal_names"].str.contains("ZTF", na=False)].copy()

ztf_df['ztf_name'] = ztf_df['internal_names'].apply(extract_ztf)
ztf_df = ztf_df[ztf_df['ztf_name'].notnull()].reset_index(drop=True)

ztf_df["objectId"] = ztf_df["ztf_name"].str.strip()
ztf_df = ztf_df[["objectId", "type","redshift"]]
ztf_df.head()

,objectId,type,redshift
0,ZTF25aafihgk,SN II,0.130000
1,ZTF21acgylcq,SN Ia,0.136700
2,ZTF25aanhiwe,SN Ia,0.092000
3,ZTF25aankqhe,SN Ia,0.052000
4,ZTF25aamrath,SN Ia,0.049751


In [4]:
print(ztf_df.shape)

(11734, 3)


In [7]:
alerce = Alerce()
root = Path("./ZTF_SN_Data")
root.mkdir(exist_ok=True)

fid_map = {1: "g", 2: "r", 3: "i"}

E404=[]
E502=[]
Eel=[]

# def robust_query(oid, retries=3, delay=1):
#     for i in range(retries):
#         try:
#             return alerce.query_lightcurve(oid, format="json")
#         except Exception as e:
#             if "404" in str(e):
#                 print(f"⚠️ No data for {oid} (404 Not Found)")
#                 return None
#             elif "502" in str(e):
#                 print(f"⚠️ 502 error on {oid}, retrying ({i+1}/{retries})...")
#                 time.sleep(delay)
#             else:
#                 print(f"❌ Unexpected error on {oid}:", e)
#                 return None
#     print(f"❌ Failed after retries: {oid}")
#     return None

for _, row in ztf_df.iterrows():
    oid = row['objectId']
    sn_type = row['type']
    redshift = row.get("redshift", "unknown")

    for i in range(3):
        try:
            lc_data = alerce.query_lightcurve(oid, format="json")
            detections = lc_data.get("detections", [])

            if not detections:
                print(f"⚠️ {oid} has no detections.")
                continue

            df = pd.DataFrame(detections)

            required_columns = ["mjd", "magpsf", "sigmapsf", "fid", "rb"]
            if not all(col in df.columns for col in required_columns):
                print(f"❌ {oid} missing one or more expected columns: {df.columns}")
                continue

            # 添加 filter 字符串列
            # df["filter"] = df["fid"].map(fid_map)

            # 确认包含 mjd 字段
            columns_to_save = ["mjd", "magpsf", "sigmapsf", "rb"]



            g_df = df[df["fid"] == 1][columns_to_save]
            r_df = df[df["fid"] == 2][columns_to_save]
            i_df = df[df["fid"] == 3][columns_to_save]

            folder = root / oid
            folder.mkdir(exist_ok=True)

            g_df.to_csv(folder / f"{oid}_g.csv", index=False)
            r_df.to_csv(folder / f"{oid}_r.csv", index=False)
            i_df.to_csv(folder / f"{oid}_i.csv", index=False)

            with open(folder / "label.txt", "w") as f:
                f.write(f"Type: {sn_type}\n")
                f.write(f"Redshift: {redshift}\n")
            
            break
            # print(f"✅ Saved: {oid}")
        except Exception as e:
            if "404" in str(e):
                # print(f"⚠️ No data for {oid} (404 Not Found)")
                E404.append(oid)
                break
            elif "502" in str(e):
                print(f"⚠️ 502 error on {oid}, retrying ({i+1}/{3})...")
                E502.append(oid)
                time.sleep(1)
            else:
                print(f"❌ Unexpected error on {oid}:", e)
                Eel.append(oid)
                break



⚠️ 502 error on ZTF22abmcfrc, retrying (1/3)...
⚠️ 502 error on ZTF22abjafpr, retrying (1/3)...
⚠️ 502 error on ZTF22abedoyq, retrying (1/3)...
⚠️ 502 error on ZTF22abivrnq, retrying (1/3)...
⚠️ 502 error on ZTF20abxrsjb, retrying (1/3)...
⚠️ 502 error on ZTF20abuojuw, retrying (1/3)...
⚠️ 502 error on ZTF20abliiex, retrying (1/3)...
⚠️ 502 error on ZTF20abkacis, retrying (1/3)...
⚠️ 502 error on ZTF20abpscln, retrying (1/3)...
⚠️ 502 error on ZTF19aawafvn, retrying (1/3)...
⚠️ 502 error on ZTF19aayjhpg, retrying (1/3)...
⚠️ 502 error on ZTF19aavxpbx, retrying (1/3)...
⚠️ 502 error on ZTF19aakzwao, retrying (1/3)...
⚠️ 502 error on ZTF19aafnend, retrying (1/3)...
⚠️ 502 error on ZTF19aakoese, retrying (1/3)...


In [17]:
lc_data = alerce.query_lightcurve("ZTF25aalyccn", format="json")
detections = lc_data.get("detections", [])
print(detections)

[{'tid': 'ztf', 'mjd': 60773.17151620006, 'candid': '3019171514015015005', 'fid': 1, 'pid': 3019171514015, 'diffmaglim': 19.437658, 'isdiffpos': 1, 'nid': 3019, 'distnr': 3.2086563, 'magpsf': 18.774733, 'magpsf_corr': None, 'magpsf_corr_ext': None, 'magap': 18.6821, 'magap_corr': None, 'sigmapsf': 0.1486874, 'sigmapsf_corr': None, 'sigmapsf_corr_ext': None, 'sigmagap': 0.2197, 'sigmagap_corr': None, 'ra': 128.0450585, 'dec': 27.4412943, 'rb': 0.9071429, 'rbversion': 't17_f5_c3', 'drb': 0.9992175, 'magapbig': 18.6288, 'sigmagapbig': 0.2682, 'rfid': 617120140, 'has_stamp': True, 'corrected': False, 'dubious': False, 'candid_alert': None, 'step_id_corr': '27.5.4', 'phase': 0.0, 'parent_candid': None}, {'tid': 'ztf', 'mjd': 60775.17874999996, 'candid': '3021178744015015012', 'fid': 2, 'pid': 3021178744015, 'diffmaglim': 19.858105, 'isdiffpos': 1, 'nid': 3021, 'distnr': 3.2516096, 'magpsf': 18.98995, 'magpsf_corr': None, 'magpsf_corr_ext': None, 'magap': 19.1147, 'magap_corr': None, 'sigmap

In [9]:
with open("Error 404.txt", "w") as f:
    for item in E404:
        f.write(item + "\n")
print("列表已写入文件。")

列表已写入文件。


In [10]:
print("Error 404:", len(E404))

Error 404: 101
